**Loading libraries**

In [ ]:
import sys
import glob
import os
import boto3
sys.path.append(os.path.split(os.getcwd())[0])
import functionsROMSv3 as clf
from datetime import datetime, timedelta
import netCDF4 as nc
import xarray as xr
import logging
import re
import numpy as np
from tqdm import tqdm
import importlib
from joblib import Parallel, delayed


**Specify your AWS credentials and region and replace these values with your own credentials.**

In [ ]:
AWS_ACCESS_KEY_ID = 'Your access key ID'
AWS_SECRET_ACCESS_KEY = 'Your secret access key'
AWS_REGION = 'us-east-1'

**Define your bucket and file information**

In [ ]:
BUCKET_NAME = 'romstest'
out_dir = f'{os.getcwd()}/'

**Setting up download region**

In [7]:
lon1 = (-116., -24.)
lat1 = (-6, 55)
depth = (0., 6000.)
serviceID = "GLOBAL_ANALYSISFORECAST_PHY_001_024"
outname = 'copernicus'

#-- Boundary dates
start_date1 = datetime(2020, 11, 30, 12)
end_date1 = datetime(2020, 12, 1, 12)

**Set up names and other things**

In [8]:
#-- Time units
time_units = 'days since 2011-01-01 00:00:00'

#-- netcdf with variables needed 
icName = 'doppio_ini_7km.nc'
gridName = 'grid_doppio_7km.nc'

#-- Names to save your files
climName1='climatology_eccofs'
bcName1 = 'bc_eccofs'

#-- Extrapolation and Interpolation options
extrapModel='gridfill' # gridfill, poisson, griddata
interpModel='basemap' # basemap, bilinear, None. The None option only works when extrapModel='griddata'

#-- Figures names
clmFig = 'surfaceInterpolation' # this is to plot surface.
bcFig = 'boundaries' # it gives the boundaries in a plot.

**Define if you want BC out of the clim file, what boundaries to extract from.**

In [9]:
createBC1='yes'
bcFlags = ['n','s','e','w']

**Define if you want storage information.**

In [10]:
showStorage='yes'

**Email settings**

In [11]:
sender_email = "email"
receiver_email = "email" 
password = "mwiwtsjkyeyelcxd"

# ----------------------      No changes needed  ----------------------

### _Sending email at the beginning to let user know preprocessing has started._

In [ ]:
subject = "This is an email from AWS to me"
body = "Hi there, this is an email to let you know that your code has just started.\
        This email was sent at "+datetime.now().strftime('%Y-%m-%d %H:%M:%S %p')+"."


clf.emailProcess(sender_email,receiver_email,subject,body,password)

### _Trying to access bucket on AWS, if it does not work, sends an email to the user._


In [ ]:
try:
    s3_client = boto3.client(
        's3',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name=AWS_REGION
    )
except Exception as e:
    body = str(e)+'. It will not be able to upload files to s3 bucket.\
           This error happened at '+datetime.now().strftime('%Y-%m-%d %H:%M:%S %p')+"."
    subject="boto3 error message"
    clf.emailProcess(sender_email,receiver_email,subject,body,password)  

### _Downloading Mercator files based on the settings above_

In [ ]:
clf.downloadingMercatorV4(outname,out_dir,serviceID, lon=lon1, lat=lat1,\
                     start_date=start_date1, end_date= end_date1,\
                    varName=['temp','salt','vel','ssh'],deltat=1,sendEmail='yes',\
                     sender_email=sender_email,receiver_email=receiver_email,\
                     password=password)

### _Creating a soft link with the Mercator files downloaded._

In [ ]:
listFiles = glob.glob('all_copernicus_yr*.nc')
listFiles.sort()

### _Getting time from the netcdf files._

In [ ]:
time = []
idx = []
for i in listFiles:
    print(i)
    file = nc.Dataset(i)
    a = re.split('[_, .]',i)
    idx.append(a[1]+'_'+a[2]+'_'+a[3]+'_'+a[4])
    time.append(nc.date2num(nc.num2date(file.variables['time'][:].data[0],\
               units=file.variables['time'].units,calendar=file.variables['time'].calendar),\
               units=time_units,calendar='gregorian'))
            
time=np.array(time)

### _Creating climatology files._

In [ ]:
clf.saveClimatology(timeInput=time,gridFile=icName,climName=climName1,\
                   units=time_units,createBC=createBC1,bcName=bcName1,bc=bcFlags)

gridd=clf.loadInfoToRead(icName,bcFlags,listFiles, bcName1=bcName1)


Parallel(n_jobs=1)(delayed(clf.process_file_v2)(t, file, gridd, 
          climName=climName1,extrapModel=extrapModel,\
          interpModel=interpModel) for t, file in enumerate(listFiles)) 

mercNClist=glob.glob('all*.nc')

### _Plot and save figures._

In [ ]:
clf.plotandSave(icName,climName1,bcName1,mercNClist,clmFig=clmFig,\
          bcFig=bcFig)

### _Submit netcdf files to bucket on AWS._

In [ ]:
OBJECT_NAME = [climName1+'.nc', bcName1+'.nc']

urlfiles = clf.urlFiletoUpload(OBJECT_NAME,BUCKET_NAME)

### _Creating text file._

In [ ]:
textfile=clf.fileName(urlfiles,fileTextName='urlDownloads.txt')

### _Sending email at the end to let user know preprocessing has finished._

In [ ]:
subject = "This is an email from AWS to me"
body = "Hi there, the files were downloaded successfully and the url is attached along with\
      the interpolation result. This email was sent at "+datetime.now().strftime('%Y-%m-%d %H:%M:%S %p')+"."

#--- email
clf.emailProcess(sender_email, receiver_email, subject, body, password,\
         image=[clmFig+".png",bcFig+".png"], textFile=textfile)